In [1]:
import cv2
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from PIL import Image

In [3]:
class AgePredictorCNN(nn.Module):
    def __init__(self):
        super(AgePredictorCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.dropout2 = nn.Dropout(0.25)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.dropout3 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc_dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout2(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout3(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc_dropout(x)
        x = self.fc2(x)
        return x


In [4]:
model_path = r'C:\Users\thano\Downloads\project 2\thanuj\age_cnn\age_predictor_cnn_new.pth'
model = AgePredictorCNN()
model.load_state_dict(torch.load(model_path))
model.eval()

C:\Users\thano\AppData\Local\Temp\ipykernel_1612\2443354828.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


AgePredictorCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.25, inplace=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout2): Dropout(p=0.25, inplace=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout3): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc_dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

In [6]:
face_cascade = cv2.CascadeClassifier('C:/Users/thano/Downloads/project 2/thanuj/face_and_eye_models\haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('C:/Users/thano/Downloads/project 2/thanuj/face_and_eye_models\haarcascade_eye.xml')

transform = transforms.Compose([
    transforms.Resize((64, 64)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  
])

down code is almost perfect


best code sofar!!!

In [7]:
import cv2
import numpy as np
import torch
from PIL import Image
import time

ages = []
frame_count = 0
max_observations = 19  
drowsiness_count = 0   
drowsiness_threshold = 5  

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6, minSize=(30, 30))
    frame_ages = []
    for (x, y, w, h) in faces:
        face_roi = gray[y + 10:y + h - 10, x + 25:x + w - 25]
        face_pil = Image.fromarray(face_roi)  
        
        face_tensor = transform(face_pil).unsqueeze(0) 
        eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.2, minNeighbors=7, minSize=(30, 30))
        if len(eyes) == 0:
            drowsiness_count += 1 
        else:
            drowsiness_count = 0 

        if drowsiness_count > drowsiness_threshold:
            with torch.no_grad():
                age_prediction = model(face_tensor)
            age = int(age_prediction.item())
            frame_ages.append(age)
            ages.append(age)  
            frame_count += 1
            cv2.rectangle(frame, (x + 20, y), (x + w - 20, y + h + 10), (255, 0, 0), 2)
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(frame, (x + ex + 20, y + ey), (x + ex + ew + 20, y + ey + eh), (0, 255, 0), 2)

        else:
            ages.clear()  
            frame_count = 0  

        time.sleep(0.01)
    if frame_count >= max_observations:
        average_age = 1.2 * np.median(ages)
        if average_age >= 40:
            average_age *= 1.3
        for (x, y, w, h) in faces:
            cv2.putText(frame, f"Median Age: {average_age:.1f}", (x + 15, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    cv2.imshow('Video - Face Detection and Age Prediction', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()